In [1]:
%cd ~/Repositories/NB10422645/


/home/ggarrett/Repositories/NB10422645


In [2]:
from src.dataset import *
from src.problems.two_dimensional.representation import NOmegaPointsScaleBasedPeriodic

def convert_rotational_eom_to_solution_series(ds, init_theta, n_t_samples, reference_time, save_parquet=True):
    mapping = NOmegaPointsScaleBasedPeriodic(n_samples=n_t_samples, n_design_points=3, init_theta=init_theta)
    
    df = ds.get_df()
    x_col = list(filter(lambda x: 'x' in x, df.columns))
    f_col = list(filter(lambda x: 'f' in x, df.columns))

    x_df = ds.get_df().loc[:,x_col]
    f_df = ds.get_df().loc[:,f_col]

    n_t_samples = n_t_samples
    n_samples = x_df.index.size
    mapping.reference_time = reference_time

    alpha_samples = np.zeros((n_samples,n_t_samples))
    omega_samples = np.zeros((n_samples,n_t_samples))
    theta_samples = np.zeros((n_samples,n_t_samples))

    for idx, row in x_df.iterrows():
        print(idx) if idx%100000==0 else None
        alpha_sample, omega_sample, theta_sample = mapping(row.values)
        alpha_samples[idx,:] = alpha_sample
        omega_samples[idx,:] = omega_sample
        theta_samples[idx,:] = theta_sample

    time_column = [f"t{i}" for i in range(n_t_samples)]
    dataframe_alpha = pd.DataFrame(data=alpha_samples, columns=time_column)
    dataframe_omega = pd.DataFrame(data=omega_samples, columns=time_column)
    dataframe_theta = pd.DataFrame(data=theta_samples, columns=time_column)
    
    ds_alpha = DataSetFX(output=f_df.values, input=dataframe_alpha.values, name=ds.name+"_series_alpha", root_dir="data")
    ds_omega = DataSetFX(output=f_df.values, input=dataframe_omega.values, name=ds.name+"_series_omega", root_dir="data")
    ds_theta = DataSetFX(output=f_df.values, input=dataframe_theta.values, name=ds.name+"_series_theta", root_dir="data")
    
    if save_parquet:
        ds_alpha.to_parquet(file_name=ds.name+"_series_alpha", root_dir="data")
        ds_omega.to_parquet(file_name=ds.name+"_series_omega", root_dir="data")
        ds_theta.to_parquet(file_name=ds.name+"_series_theta", root_dir="data")
        
    return ds_alpha, ds_omega, ds_theta

In [3]:
ds = DataSetFX.from_parquet(root_dir="data", file_name="dataSet_gridSampled_3.parquet")
print(len(ds))

reference_time = 6529.276071694349
n_t_samples = 652
init_theta = -3.141592653589793

ds_alpha, ds_omega, ds_theta = convert_rotational_eom_to_solution_series(ds, init_theta, n_t_samples, reference_time)

2187
0


### Args:
- in_channels (int): Number of channels in the input image
- out_channels (int): Number of channels produced by the convolution
- kernel_size (int or tuple): Size of the convolving kernel
- stride (int or tuple, optional): Stride of the convolution. Default: 1
- padding (int or tuple, optional): Zero-padding added to both sides of the input. Default: 0
- padding_mode (string, optional). Accepted values `zeros` and `circular` Default: `zeros`
- dilation (int or tuple, optional): Spacing between kernel elements. Default: 1
- groups (int, optional): Number of blocked connections from input channels to output channels. 

In [4]:
from src.surrogate.modules import *
from src.surrogate.deeplearning.core import *

### Test case 1

transform

Data:
- F = F, X = Theta
- 



In [5]:
from sklearn import preprocessing

transform = TransformDataFrame.from_ordered_dict(OrderedDict([
    ("tf1", (preprocessing.MinMaxScaler(feature_range=(0, 1)), ["f1", "f2", "f3", "f4"]))
#     ("tx1", (preprocessing.StandardScaler(copy=True, with_mean=True, with_std=True), [f"x{i}" for i in range(652)])),
]
))

ds_t1_theta = DataSetFSeries.from_parquet(root_dir="data", file_name="dataSet_gridSampled_6_series_theta", transform=transform)

# Define training function.
def train_model(name, ds, model, model_folder, load=False):
    trainer = Trainer(model_folder, ds, model, torch.optim.Adam, torch.nn.L1Loss, num_epochs=50, batch_size=int(2e4), learning_rate=1e-3)
#     trainer.add_scheduler(torch.optim.lr_scheduler.MultiStepLR, milestones=[10], gamma=0.1)
    if load:
        trainer.load_checkpoint(model_folder)
    trainer.train()

class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1, 1, 1)


In [6]:
import torch
print(torch.cuda.is_available())
name = "experiment1_series"
folder = f"models/{name}"

ds = ds_t1_theta

model1 = nn.Sequential(
    BasicBlock(input_size=1, output_size=100, kernel_size=(1, 109)),
    torch.nn.MaxPool2d((1, 109), stride=1, padding=0, dilation=1, return_indices=False, ceil_mode=False),
    BasicBlock(input_size=100, output_size=10, kernel_size=(1, 109)),
    Flatten(),
    RegressionOutput(3280, 4)
)

train_model(name, ds, model1, folder, False)


False
Interrupted


SystemExit: 0

/home/ggarrett/anaconda3/envs/sigh/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
